# 사진 자동 정리하기

사진 정보를 이용하여 위치, 날짜등의 이름으로 사진의 이름을 변경하여 정리합니다. 

[정리 전]

KakaoTalk_2022-05-06 14-58-50.jpg


[정리 후]

시흥시_은행동_2022-06-04 14-06-00.jpg


---

개발 프로세스

1. 사진에서 정보 추출
    * 위도, 경도(GPS 정보 추출) 추출
    * 시간
2. 위도, 경도 -> 주소로 변환
3. 파일이름 변경
    * 변경전:KakaoTalk_2022-05-20 14-22-57.jpg
    * 변경후:아산시_구령리_2022-05-20 14-22-57.jpg

----

## 사진 정보 추출하기

In [1]:
from PIL import Image #파이썬 이미지 라이브러리(Python Image Library)
from PIL.ExifTags import TAGS

image = Image.open(r'./사진/KakaoTalk_2022-06-04 14-06-00.jpg')
info = image._getexif() #이미지 메타정보
info

{34853: {0: b'\x02\x02\x00\x00',
  2: (37.0, 26.0, 41.294399),
  4: (126.0, 47.0, 53.931119),
  6: 52.0},
 296: 2,
 34665: 262,
 271: 'samsung',
 272: 'SM-S908N',
 305: 'S908NKSU1AVE4',
 306: '2022:06:04 14:06:00',
 531: 2,
 532: (0.0, 255.0, 128.0, 255.0, 128.0, 255.0),
 282: 150.0,
 283: 150.0,
 36864: b'0221',
 37121: b'\x01\x02\x03\x00',
 37377: 7.55,
 36867: '2022:06:04 14:06:00',
 36868: '2022:06:04 14:06:00',
 37378: 2.27,
 37379: 5.96,
 37380: 0.0,
 37381: 2.27,
 37383: 2,
 37385: 0,
 37386: 3.8,
 40961: 1,
 40962: 841,
 41989: 25,
 41990: 0,
 40963: 1121,
 33434: 0.005319148936170213,
 33437: 2.2,
 34850: 2,
 34855: 50,
 41986: 0,
 40960: b'0100',
 41987: 0,
 41988: 1.23}

In [2]:
type(info)
len(info)

36

In [3]:
#파일 속성정보 출력
# https://exiftool.org/TagNames/EXIF.html
from PIL.ExifTags import TAGS
for tag, value in info.items():
    #print(tag, value)
    # 태그숫자를 디코딩 
    decoded = TAGS.get(tag, tag)
    print(tag, decoded, value)

34853 GPSInfo {0: b'\x02\x02\x00\x00', 2: (37.0, 26.0, 41.294399), 4: (126.0, 47.0, 53.931119), 6: 52.0}
296 ResolutionUnit 2
34665 ExifOffset 262
271 Make samsung
272 Model SM-S908N
305 Software S908NKSU1AVE4
306 DateTime 2022:06:04 14:06:00
531 YCbCrPositioning 2
532 ReferenceBlackWhite (0.0, 255.0, 128.0, 255.0, 128.0, 255.0)
282 XResolution 150.0
283 YResolution 150.0
36864 ExifVersion b'0221'
37121 ComponentsConfiguration b'\x01\x02\x03\x00'
37377 ShutterSpeedValue 7.55
36867 DateTimeOriginal 2022:06:04 14:06:00
36868 DateTimeDigitized 2022:06:04 14:06:00
37378 ApertureValue 2.27
37379 BrightnessValue 5.96
37380 ExposureBiasValue 0.0
37381 MaxApertureValue 2.27
37383 MeteringMode 2
37385 Flash 0
37386 FocalLength 3.8
40961 ColorSpace 1
40962 ExifImageWidth 841
41989 FocalLengthIn35mmFilm 25
41990 SceneCaptureType 0
40963 ExifImageHeight 1121
33434 ExposureTime 0.005319148936170213
33437 FNumber 2.2
34850 ExposureProgram 2
34855 ISOSpeedRatings 50
41986 ExposureMode 0
40960 FlashPi

In [4]:
# GPSInfo 정보 추출

#파일 속성정보 출력
# https://exiftool.org/TagNames/EXIF.html
from PIL.ExifTags import TAGS
for tag, value in info.items():
    #print(tag, value)
    # 태그숫자를 디코딩 
    decoded = TAGS.get(tag, tag)
    #print(tag, decoded, value)
    if decoded == 'GPSInfo':
        print(f'{decoded}:{value}')
# 출력

# GPSInfo {0: b'\x02\x02\x00\x00', 2: (37.0, 26.0, 41.294399), 4: (126.0, 47.0, 53.931119), 6: 52.0}

GPSInfo:{0: b'\x02\x02\x00\x00', 2: (37.0, 26.0, 41.294399), 4: (126.0, 47.0, 53.931119), 6: 52.0}


In [5]:
# GPSInfo 정보 추출(위도, 경도)
for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    if decoded == 'GPSInfo':
        gps_lat = value.get(2)# 위도
        gps_lon = value.get(4)# 경도

print(gps_lat)
print(gps_lon)

(37.0, 26.0, 41.294399)
(126.0, 47.0, 53.931119)


In [6]:
# 위도 경도 값 보정 (도,분,초) 1도는 60분이며, 1분은 60초이다. 1도 = 60분 = 3600초

lat = (((gps_lat[2] / 60.0 ) +  gps_lat[1]) / 60.0) + gps_lat[0]
lon = (((gps_lon[2] / 60.0 ) +  gps_lon[1]) / 60.0) + gps_lon[0]

lat, lon

(37.444803999722225, 126.79831419972223)

## 날짜 정보 추출하기

In [17]:
# 날짜 포맷을 2023-07-20 13-37-00으로 추출
import datetime

for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    if decoded == 'DateTimeOriginal':
        dt = datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
        day = dt.strftime('%Y-%m-%d %H-%M-%S')
print(day)
        

2022-06-04 14-06-00


## 위도,경도에서 주소로 변환

https://pypi.org/project/geopy/

In [18]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/119.8 kB ? eta -:--:--
     ----------------------------- --------- 92.2/119.8 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 119.8/119.8 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
from geopy.geocoders import Nominatim

In [21]:
# 지오코드 사용법 살펴보기
# 주소를 위도 경도로 변환

from geopy.geocoders import Nominatim

test = '구로구 디지털로 27'
# test = '강남구 학동로 346'
geolocator = Nominatim(user_agent='South Korea') # user_agent는 식별가능 문자열(아무 문자열)
location = geolocator.geocode(test)

latitude = location.latitude #위도
longitude = location.longitude #경도

print(latitude,longitude )

37.4873082 126.8975357


In [22]:
# 위도 경도를 주소로 변환
lat_lng_str = f'{latitude},{longitude}' #위도 경도 문자열
loc = geolocator.reverse(lat_lng_str)
loc.address

'현대아파트, 도림천로, 구로3동, 구로구, 서울, 08378, 대한민국'

In [24]:
lat_lng_str = '37.444803999722225, 126.79831419972223' #위도 경도 문자열
loc = geolocator.reverse(lat_lng_str)
loc.address

'맥도날드, 대은로, 대야동, 은행동, 시흥시, 경기도, 대한민국'

---

## 함수 생성

In [1]:
# 사진 정보 가져오기
# 매개변수: 파일 이름
# 반환값: 위도, 경도, 시간
def getPicInfo(fname):
    image = Image.open(fname)
    info = image._getexif()
    image.close()

    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        if decoded == 'GPSInfo':
            gps_lat = value.get(2)# 위도
            gps_lon = value.get(4)# 경도

            lat = (((gps_lat[2] / 60.0 ) +  gps_lat[1]) / 60.0) + gps_lat[0]
            lon = (((gps_lon[2] / 60.0 ) +  gps_lon[1]) / 60.0) + gps_lon[0]
        
        if decoded == 'DateTimeOriginal':
            dt = datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
            day = dt.strftime('%Y-%m-%d %H-%M-%S')
    
    return lat, lon, day
    

        

In [27]:
getPicInfo('./사진/KakaoTalk_2022-06-04 14-06-00.jpg')

(37.444803999722225, 126.79831419972223, '2022-06-04 14-06-00')

In [2]:
# 주소 가져오기
def getAddress(latitude,longitude):
    lat_lng_str = f'{latitude},{longitude}' #위도 경도 문자열
    
    geolocator = Nominatim(user_agent='South Korea')

    loc = geolocator.reverse(lat_lng_str)  
    addlist = loc.address.split(',')

    si = addlist[3].strip()
    do = addlist[2].strip()

    return si, do
    

In [31]:
getAddress(37.444803999722225, 126.79831419972223)

('은행동', '대야동')

## 종합코드

In [6]:
from PIL import Image #파이썬 이미지 라이브러리(Python Image Library)
from PIL.ExifTags import TAGS
from geopy.geocoders import Nominatim
from glob import glob
import os
import datetime

flist = glob(r'사진\*.jpg')

for fname in flist:

    lat, lon, pictime = getPicInfo(fname)
    si,do = getAddress(lat,lon)
    # print(lat, lon, pictime)
    # print(si,do)
    newfname = f'사진\\{si}_{do}_{pictime}.jpg'
    # print(newfname)

    os.rename(fname,newfname)
    
    print(fname)

사진\아산시_구령리_2022-05-20 14-22-57.jpg
사진\아산시_백암리_2022-05-06 14-58-50.jpg
사진\은행동_대야동_2022-06-04 14-06-00.jpg
사진\충청남도_아산시_2022-06-01 15-52-35.jpg


##에러가 발생하는 경우

In [11]:
import random
if (random.randint(0,1)):
    raise Exception ('나는 잼민이 에러ㅋㅋㅋㅋ')

In [13]:
# 사진 정보 가져오기
# 매개변수: 파일 이름
# 반환값: 위도, 경도, 시간
def getPicInfo(fname):
    image = Image.open(fname)
    info = image._getexif()
    image.close()

    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        if decoded == 'GPSInfo':
            gps_lat = value.get(2)# 위도
            gps_lon = value.get(4)# 경도

            lat = (((gps_lat[2] / 60.0 ) +  gps_lat[1]) / 60.0) + gps_lat[0]
            lon = (((gps_lon[2] / 60.0 ) +  gps_lon[1]) / 60.0) + gps_lon[0]
        
        if decoded == 'DateTimeOriginal':
            dt = datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
            day = dt.strftime('%Y-%m-%d %H-%M-%S')
    
    return lat, lon, day
    

In [12]:
# 주소 가져오기
def getAddress(latitude,longitude):
    lat_lng_str = f'{latitude},{longitude}' #위도 경도 문자열
    
#####################################################################
# 에러 랜덤 발생
#####################################################################
    import random
    if (random.randint(0,1)):
        raise Exception ('나는 잼민이 에러ㅋㅋㅋㅋ')        
    
    
    geolocator = Nominatim(user_agent='South Korea')

    loc = geolocator.reverse(lat_lng_str)  
    addlist = loc.address.split(',')

    si = addlist[3].strip()
    do = addlist[2].strip()

    return si, do
    

In [16]:
from PIL import Image #파이썬 이미지 라이브러리(Python Image Library)
from PIL.ExifTags import TAGS
from geopy.geocoders import Nominatim
from glob import glob
import os
import datetime

flist = glob(r'사진\*.jpg')

for fname in flist:

    lat, lon, pictime = getPicInfo(fname)
    try:
        si,do = getAddress(lat,lon)
    # print(lat, lon, pictime)
    # print(si,do)
    except Exception:
        print(f'{fname}에서 에러남')
        pass

    newfname = f'사진\\{si}_{do}_{pictime}.jpg'
    # print(newfname)

    os.rename(fname,newfname)

    
    
    print(fname)

사진\아산시_구령리_2022-05-20 14-22-57.jpg
사진\아산시_백암리_2022-05-06 14-58-50.jpg에서 에러남
사진\아산시_백암리_2022-05-06 14-58-50.jpg
사진\아산시_백암리_2022-06-04 14-06-00.jpg
사진\충청남도_아산시_2022-06-01 15-52-35.jpg에서 에러남
사진\충청남도_아산시_2022-06-01 15-52-35.jpg
